In [ ]:
# 방사형 차트를 그리는 함수 (수치 안쪽으로 배치)
def plot_radar_chart_pentagon(store_name, data_pig, data_ratios, save_path="./data/방사형차트/"):
    import os
    # 저장 경로 폴더 생성
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # 해당 가게 데이터 가져오기
    pig_info = data_pig[data_pig['store_name'] == store_name].iloc[0]
    ratio_info = data_ratios[data_ratios['store_name'] == store_name].iloc[0]
    
    # 데이터 준비 (비율 계산)
    categories = ['맛', '서비스', '양', '별점', '가격']
    
    # 4개 가게의 평균 가격 계산
    avg_price = data_pig['price_range'].mean()
    
    values = [
        ratio_info['final_taste_positive'],  # 맛: 긍정 비율
        ratio_info['final_service_positive'],  # 서비스: 긍정 비율
        ratio_info['final_quantity_positive'],  # 양: 긍정 비율
        pig_info['weighted_rating'] / 5,  # 별점: 1~5점을 0~1로 정규화
        1 if pig_info['price_range'] <= avg_price else 1 - ((pig_info['price_range'] - avg_price) / avg_price)  # 가격 점수 (평균 기준)
    ]
    values += values[:1]  # 닫힌 폴리곤

    # 각도 계산
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]

    # 플롯 생성
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.plot(angles, values, label=store_name, linewidth=2, linestyle='solid')
    ax.fill(angles, values, alpha=0.25)

    # 각 점에 값 표시 (안쪽으로 배치)
    for i, (angle, value, category) in enumerate(zip(angles, values, categories + [categories[0]])):
        # 값 표시 위치 조정 (안쪽으로 이동)
        adjustment = 0.1  # 중심으로 이동할 정도
        ax.text(
            angle,
            value - adjustment,  # 안쪽으로 값 이동
            f"{value:.2f}",
            horizontalalignment='center',
            verticalalignment='center',
            fontsize=10,
            color='black'
        )

    # 설정 (오각형 그리드 추가)
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_thetagrids(np.degrees(angles[:-1]), categories)
    ax.set_ylim(0, 1)  # 비율 기반으로 0~1 범위 설정

    # 오각형 그리드 설정
    num_levels = 5  # 오각형 그리드 단계
    grid_values = np.linspace(0, 1, num_levels + 1)[1:]  # 0을 제외한 값
    for value in grid_values:
        ax.plot(angles, [value] * len(angles), color='gray', linestyle='dotted', linewidth=0.8, alpha=0.6)

    # 외곽 원 제거
    ax.spines['polar'].set_visible(False)
    ax.set_yticks([])  # y축 라벨 제거
    ax.set_title(f"{store_name} 리뷰 분석", size=15, pad=20)

    # 이미지 저장
    file_name = f"{store_name}_radar_chart.png"
    file_path = os.path.join(save_path, file_name)
    plt.savefig(file_path, dpi=300, bbox_inches='tight')  # 고해상도로 저장
    plt.close(fig)  # 플롯 닫기
    print(f"{store_name} 차트가 {file_path}에 저장되었습니다.")

# 각 가게별 방사형 차트 생성 및 저장
for store in target_stores:
    plot_radar_chart_pentagon(store, filtered_pig_data, filtered_ratios)
